In [ ]:
!pip install torch

In [3]:
device = torch.device("mps")

In [1]:
import sys
# sys.path.append('..')

import torch

In [2]:
x = torch.arange(15, dtype=torch.float64).view(5, 3)
## 相当于生成0～15的数组，然后扭成5*3的矩阵。

In [3]:
x

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]], dtype=torch.float64)

In [4]:
gamma = torch.ones(x.shape[1]) ## 方差？
beta = torch.zeros(x.shape[1]) ## 均值？
## 跟列的形状保持一致。

In [5]:
gamma

tensor([1., 1., 1.])

In [6]:
beta

tensor([0., 0., 0.])

In [7]:
x_mean = torch.mean(x, dim=0, keepdim=True)
## 如果我们直接照搬原来的代码，是跑不了的。
## 这边求平均的时候，说，只支持数据格式为float。
## 呵呵哒。
## 这里就是在行这个维度求平均。或者说，取一列出来，求平均。
## 呵呵哒。
x_mean

tensor([[6., 7., 8.]], dtype=torch.float64)

In [8]:
x - x_mean
## 这里的减法操作，相当于每一行各自相减。
## 这个操作是我没有想到的。
## 两个相减的张量，形状都不一样，怎么会能相减呢？
## 这样看来，还真是出乎我的意料啊。

tensor([[-6., -6., -6.],
        [-3., -3., -3.],
        [ 0.,  0.,  0.],
        [ 3.,  3.,  3.],
        [ 6.,  6.,  6.]], dtype=torch.float64)

In [9]:
x_var = torch.mean((x - x_mean)**2, dim=0, keepdim=True)
## 这里的平方，也是每一个元素各自平方。
## 所以如果要矩阵相乘，可能得用torch.mm之类的方法？

In [10]:
x_var

tensor([[18., 18., 18.]], dtype=torch.float64)

In [11]:
eps = 1e-5

x_hat = (x - x_mean) / torch.sqrt(x_var + eps)

x_hat
## 这个就是批归一化的结果了。

tensor([[-1.4142, -1.4142, -1.4142],
        [-0.7071, -0.7071, -0.7071],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.7071,  0.7071,  0.7071],
        [ 1.4142,  1.4142,  1.4142]], dtype=torch.float64)

In [12]:
## 所以上述的操作，可以理解成：
## batch中，在某个维度上的所有值进行normalization。然后各个维度独立开展这样的操作。

In [13]:
x_mean

tensor([[6., 7., 8.]], dtype=torch.float64)

In [16]:
gamma

tensor([1., 1., 1.])

In [17]:
gamma.view_as(x_mean) 
## 看起来就是将 gamma 改变成 x_mean 的形状。

tensor([[1., 1., 1.]])

In [18]:
x_hat

tensor([[-1.4142, -1.4142, -1.4142],
        [-0.7071, -0.7071, -0.7071],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.7071,  0.7071,  0.7071],
        [ 1.4142,  1.4142,  1.4142]], dtype=torch.float64)

In [19]:
(gamma.view_as(x_mean)*2) * x_hat \ ## 这里我感觉好像是在说ho，gamma原来是一维的，现在改成二维的。然后跟x_hat对应的维度的，进行对应位置相乘。
+ beta.view_as(x_mean)

tensor([[-2.8284, -2.8284, -2.8284],
        [-1.4142, -1.4142, -1.4142],
        [ 0.0000,  0.0000,  0.0000],
        [ 1.4142,  1.4142,  1.4142],
        [ 2.8284,  2.8284,  2.8284]], dtype=torch.float64)

In [21]:
import numpy as np
from torchvision.datasets import mnist # 导入 pytorch 内置的 mnist 数据
from torch.utils.data import DataLoader
from torch import nn
from torch.autograd import Variable

In [24]:
nn.ReLU?